## Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import os
from google.colab import files
from sqlalchemy import create_engine
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/BI Course/Project
%ls

/content/drive/MyDrive/BI Course/Project
FactCustomerPurchase.csv       olist_order_payments_dataset.csv
FactCustomerPurchase.json      olist_order_reviews_dataset.csv
FactSales.json                 olist_orders_dataset.csv
olist_customers_dataset.csv    olist_products_dataset.csv
olist_geolocation_dataset.csv  olist_sellers_dataset.csv
olist_order_items_dataset.csv  product_category_name_translation.csv


# Extract

##Read CSV From Directory

In [ ]:
Customers_DB = pd.read_csv("olist_customers_dataset.csv")
Sellers_DB = pd.read_csv("olist_sellers_dataset.csv")
OrdersReviews_DB = pd.read_csv("olist_order_reviews_dataset.csv")
OrderItems_DB = pd.read_csv("olist_order_items_dataset.csv")
Products_DB = pd.read_csv("olist_products_dataset.csv")
Locations_DB = pd.read_csv("olist_geolocation_dataset.csv")
ProductCategory_DB = pd.read_csv("product_category_name_translation.csv")
Orders_DB = pd.read_csv("olist_orders_dataset.csv")
Payments_DB = pd.read_csv("olist_order_payments_dataset.csv")

# Transform

##Create Dim Tables 

Extract only interested fields and tables

In [ ]:
DimSellers=Sellers_DB[['seller_id','seller_zip_code_prefix']]
DimLocation=Locations_DB[['geolocation_zip_code_prefix','geolocation_state']]
DimCustomer=Customers_DB[['customer_id','customer_zip_code_prefix']]
DimItems=OrderItems_DB[['order_id','order_item_id','product_id','seller_id','price','freight_value']]
DimProducts=Products_DB[['product_id','product_category_name']]
DimOrder=Orders_DB.join(OrdersReviews_DB.set_index('order_id'), on='order_id', how='left')\
.join(Payments_DB.set_index('order_id'), on='order_id', how='left')\
[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp','order_delivered_customer_date','review_score','payment_type','payment_value']]

##Create Fact Tables





###Create FactSales




In [ ]:
FactSales = DimItems.join(DimSellers.set_index('seller_id'), on='seller_id', how='right').join(DimOrder.set_index('order_id'), on='order_id', how='left')
FactSales.groupby(by=['seller_id','order_id','customer_id'],as_index=False).count()
FactSales.rename(columns={"order_item_id": "amount_of_sales_items"},inplace=True)
FactSales['amount_of_sales'] = 1
FactSales['amount_of_customers'] = 1
FactSales = FactSales[['seller_id','order_id','customer_id','amount_of_sales_items','amount_of_sales','amount_of_customers']].reset_index(drop = True)
FactSales

,seller_id,order_id,customer_id,amount_of_sales_items,amount_of_sales,amount_of_customers
0,3442f8959a84dea7ee197c632cb2df15,4a90af3e85dd563884e2afeab1091394,9d6837f9700a3441e7183bff3bc4eef0,1,1,1
1,3442f8959a84dea7ee197c632cb2df15,6d953888a914b67350d5bc4d48f2acab,a973c4e3ad82777add3fa188f91dacea,1,1,1
2,3442f8959a84dea7ee197c632cb2df15,bc8a5de6abf5b14f98a6135a7fb46731,1554ffe702931a062b4383b109accf63,2,1,1
3,d1b65fc7debc3361ea86b5f14c68d2e2,035201c3c82a97f8a25dd6bd5244b2d5,9facbfd2dd51a45404d58154b12ed2dd,1,1,1
4,d1b65fc7debc3361ea86b5f14c68d2e2,0504447548229e075dea8441b37b1e2a,4e2c1f15de98416a90c2ee06b55ccc9b,1,1,1
...,...,...,...,...,...,...
118305,e603cf3fec55f8697c9059638d6c8eb5,7aa50aad959e4345b45b9732bc346a6e,f0426deff141dffab4da4832186f4a36,1,1,1
118306,e603cf3fec55f8697c9059638d6c8eb5,aae373a2cc64ed2d1f262aa56e70b7e8,63d8ce2f854381279f3b762e962dc969,1,1,1
118307,e603cf3fec55f8697c9059638d6c8eb5,bc34d5b185152cb88b9f5db391dcd1cb,9a3aaf9447f2fbd34372e69398b2ede3,1,1,1
118308,e603cf3fec55f8697c9059638d6c8eb5,d1edb80f6670902ebbb11765b96873da,4d3e75dab77ee14de42b66f557ea4d3a,1,1,1


###Create FactCustomerPurchase


In [ ]:
FactCustomerPurchase=DimOrder.join(DimItems.set_index('order_id'), on='order_id', how='left')
FactCustomerPurchase_sum = FactCustomerPurchase.groupby(by=['customer_id','order_id','product_id'],as_index=False).sum()[['customer_id','order_id','product_id','payment_value']]
FactCustomerPurchase_count = FactCustomerPurchase.groupby(by=['customer_id','order_id','product_id'],as_index=False).count()[['customer_id','order_id','product_id','order_item_id']]
FactCustomerPurchase_count.rename(columns={"order_item_id": "amount_of_items_purchase"},inplace=True)
FactCustomerPurchase = pd.merge(FactCustomerPurchase_count, FactCustomerPurchase_sum,  how='inner', left_on=['customer_id','order_id','product_id'], right_on = ['customer_id','order_id','product_id'])
FactCustomerPurchase['amount_of_orders'] = 1
FactCustomerPurchase.rename(columns={"payment_value": "total_spend"},inplace=True)
FactCustomerPurchase[['customer_id','order_id','product_id','amount_of_items_purchase','amount_of_orders','total_spend']]
FactCustomerPurchase

,customer_id,order_id,product_id,amount_of_items_purchase,total_spend,amount_of_orders
0,00012a2ce6f8dcda20d059ce98491703,5f79b5b0931d63f1a42989eb65b9da6e,64315bd8c0c47303179dd2e25b579d00,1,114.74,1
1,000161a058600d5901f007fab4c27140,a44895d095d7e0702b6a162fa2dbeced,84183944dc7cddca87a5d384452c1d3c,1,67.41,1
2,0001fd6190edaaf884bcaf3d49edf079,316a104623542e4d75189bb372bc5f8d,9df2b21ec85378d71df4404712e17478,1,195.42,1
3,0002414f95344307404f0ace7a26f1d5,5825ce2e88d5346438686b0bba99e5ee,af3ec22cce878225aae6d9eb6c7a78eb,1,179.35,1
4,000379cdec625522490c315e70c7a9fb,0ab7fb08086d4af9141453c91878ed7a,868b3136c5b206f91b8208fbfdf2cb7c,1,107.01,1
...,...,...,...,...,...,...
102420,fffcb937e9dd47a13f05ecb8290f4d3e,620ddc9fbe3e445676a8e71d4a830821,2eb705094d8383cfa50e8965dddc8a7e,1,91.91,1
102421,fffecc9f79fd8c764f843e9951b11341,814d6a3a7c0b32b2ad929ac6328124e9,c045b19fdb30a3035c56d04c6fe2e622,3,81.36,1
102422,fffeda5b6d849fbd39689bb92087f431,8c855550908247a7eff50281b92167a8,d5c263f4d651ba657c900ce70a4be7de,1,63.13,1
102423,ffff42319e9b2d713724ae527742af25,83b5fc912b2862c5046555ded1483ae9,e61c78a7343d82c0539d27df0f7dfc31,1,214.13,1


#Load

In [ ]:
FactCustomerPurchase.to_json('FactCustomerPurchase.json', orient='records')
FactSales.head(15000).to_json('FactSales.json', orient='records')
#files.download('FactSales.json')
#files.download('FactCustomerPurchase.json')


FactCustomerPurchase.head(15000).to_csv('FactCustomerPurchase.csv', index=False)
#FactSales.to_csv('FactSales.csv', index=False)
#files.download('FactSales.csv')
files.download('FactCustomerPurchase.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import sqlite3
sql_connection = sqlite3.connect('./bi_project.db') #create db
FactSales.to_sql('FactSales', sql_connection, if_exists='replace', index=False)
FactCustomerPurchase.to_sql('FactCustomerPurchase', sql_connection, if_exists='replace', index=False)
DimSellers.to_sql('DimSellers', sql_connection, if_exists='replace', index=False)
DimLocation.to_sql('DimLocation', sql_connection, if_exists='replace', index=False)
DimCustomer.to_sql('DimCustomer', sql_connection, if_exists='replace', index=False)
DimItems.to_sql('DimItems', sql_connection, if_exists='replace', index=False)
DimProducts.to_sql('DimProducts', sql_connection, if_exists='replace', index=False)
DimOrder.to_sql('DimOrder', sql_connection, if_exists='replace', index=False)

In [ ]:
cursor = sql_connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print('Name of tables:',cursor.fetchall())


Name of tables: [('FactSales',), ('FactCustomerPurchase',), ('DimSellers',), ('DimLocation',), ('DimCustomer',), ('DimItems',), ('DimProducts',), ('DimOrder',)]


In [ ]:
pd.read_sql('SELECT * FROM FactSales', con = sql_connection).head()

,seller_id,order_id,customer_id,amount_of_sales_items,amount_of_sales,amount_of_customers
0,3442f8959a84dea7ee197c632cb2df15,4a90af3e85dd563884e2afeab1091394,9d6837f9700a3441e7183bff3bc4eef0,1,1,1
1,3442f8959a84dea7ee197c632cb2df15,6d953888a914b67350d5bc4d48f2acab,a973c4e3ad82777add3fa188f91dacea,1,1,1
2,3442f8959a84dea7ee197c632cb2df15,bc8a5de6abf5b14f98a6135a7fb46731,1554ffe702931a062b4383b109accf63,2,1,1
3,d1b65fc7debc3361ea86b5f14c68d2e2,035201c3c82a97f8a25dd6bd5244b2d5,9facbfd2dd51a45404d58154b12ed2dd,1,1,1
4,d1b65fc7debc3361ea86b5f14c68d2e2,0504447548229e075dea8441b37b1e2a,4e2c1f15de98416a90c2ee06b55ccc9b,1,1,1


##Sql Queries